<img src="https://github.com/NTMC-Community/MatchZoo/blob/2.0/artworks/matchzoo-logo.png?raw=True" alt="logo" style="width:600px;float: center"/>

In [1]:
import matchzoo as mz

Using TensorFlow backend.


# Prepare Data

In [57]:
train_data_pack = mz.datasets.wiki_qa.load_data(stage='train', task='ranking')
test_data_pack = mz.datasets.wiki_qa.load_data(stage='test', task='ranking')

In [58]:
type(train_data_pack)

matchzoo.data_pack.data_pack.DataPack

`DataPack` is a MatchZoo native data structure that most MatchZoo data handling processes build upon. A `DataPack` is consists of three `pandas.DataFrame`:

In [59]:
train_data_pack.left.head()

,text_left
id_left,
Q1,how are glacier caves formed?
Q2,How are the directions of the velocity and for...
Q5,how did apollo creed die
Q6,how long is the term for federal judges
Q7,how a beretta model 21 pistols magazines works


In [60]:
train_data_pack.right.head()

,text_right
id_right,
D1-0,A partly submerged glacier cave on Perito More...
D1-1,The ice facade is approximately 60 m high
D1-2,Ice formations in the Titlis glacier cave
D1-3,A glacier cave is a cave formed within the ice...
D1-4,"Glacier caves are often called ice caves , but..."


In [61]:
train_data_pack.relation.head()

,id_left,id_right,label
0,Q1,D1-0,0
1,Q1,D1-1,0
2,Q1,D1-2,0
3,Q1,D1-3,1
4,Q1,D1-4,0


It is also possible to convert a `DataPack` into a single `pandas.DataFrame` that holds all information.

In [62]:
train_data_pack.frame().head()

,id_left,text_left,id_right,text_right,label
0,Q1,how are glacier caves formed?,D1-0,A partly submerged glacier cave on Perito More...,0
1,Q1,how are glacier caves formed?,D1-1,The ice facade is approximately 60 m high,0
2,Q1,how are glacier caves formed?,D1-2,Ice formations in the Titlis glacier cave,0
3,Q1,how are glacier caves formed?,D1-3,A glacier cave is a cave formed within the ice...,1
4,Q1,how are glacier caves formed?,D1-4,"Glacier caves are often called ice caves , but...",0


However, using such `pandas.DataFrame` consumes much more memory if there are many duplicates in the texts, and that is the exact reason why we use `DataPack`. For more details about data handling, consult `matchzoo/tutorials/data_handling.ipynb`.

# Preprocessing

MatchZoo preprocessors are used to convert a raw `DataPack` into a `DataPack` that ready to be fed into a model. 

In [63]:
preprocessor = mz.preprocessors.NaivePreprocessor()

There are two steps to use a preprocessor. First, `fit`. Then, `transform`. `fit` will only changes the preprocessor's inner state but not the input `DataPack`.

In [64]:
preprocessor.fit(train_data_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2117/2117 [00:00<00:00, 10113.17it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18828/18828 [00:03<00:00, 4999.25it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 418540/418540 [00:00<00:00, 2473958.72it/s]


`fit` will gather all information it needs into its `context`. In the above example, we can see a `VocabularyUnit` is built during the fitting process using `train_data_pack`.

In [65]:
preprocessor.context

{'vocab_unit': <matchzoo.processor_units.processor_units.VocabularyUnit at 0x12ce30fd0>}

`VocabularyUnit` is a `StatefulProcessorUnit` that has a similar `fit`/`transform` interface. Once a `VocabularyUnit` `fit`, it will store a mapping from `term` to `index` and the reverse in its `state`.

The `NaivePreprocessor` already handles `VocabularyUnit` internally, so we do not have to worry about that. Just access it through the `NaivePreprocessor`'s `context`.

In [66]:
vocab_unit = preprocessor.context['vocab_unit']
print(vocab_unit.state['term_index']['match'])
print(vocab_unit.state['term_index']['zoo'])
print(vocab_unit.state['index_term'][1])
print(vocab_unit.state['index_term'][2])

8783
1111
61
rapid


Once `fit`, the preprocessor has enough information to `transform`.  `transform` will not change the preprocessor's inner state and the input `DataPack`, but return a transformed `DataPack`.

In [67]:
train_data_pack_processed = preprocessor.transform(train_data_pack)
test_data_pack_processed = preprocessor.transform(test_data_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => VocabularyUnit => FixedLengthUnit: 100%|██████████| 2117/2117 [00:00<00:00, 8024.41it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => VocabularyUnit => FixedLengthUnit: 100%|██████████| 18828/18828 [00:04<00:00, 4681.04it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => VocabularyUnit => FixedLengthUnit: 100%|██████████| 630/630 [00:00<00:00, 8012.50it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => VocabularyUnit => FixedLengthUnit: 100%|██████████| 5914/5914 [00:01<00:00, 4581.09it/s]


In [68]:
train_data_pack_processed.left.head()

,text_left
id_left,
Q1,"[12865, 29105, 964, 4922, 22693, 0, 0, 0, 0, 0..."
Q2,"[12865, 29105, 15189, 4952, 3835, 15189, 833, ..."
Q5,"[12865, 17587, 25589, 25294, 794, 0, 0, 0, 0, ..."
Q6,"[12865, 4818, 13106, 15189, 19429, 24964, 7481..."
Q7,"[12865, 4851, 13642, 29984, 2429, 23111, 15512..."


As we can see, `text_left` is already in sequence form that nerual networks love.

Just to make sure we have the correct sequence:

In [69]:
print('Before:', train_data_pack.left.loc['Q1']['text_left'])
sequence = train_data_pack_processed.left.loc['Q1']['text_left']
print('After:', sequence)
print('Translated:', '_'.join([vocab_unit.state['index_term'][i] for i in sequence]))

Before: how are glacier caves formed?
After: [12865, 29105, 964, 4922, 22693, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Translated: how_are_glacier_caves_formed_________________________


For more details about data handling, consult `matchzoo/tutorials/preprocessing.ipynb`.

# Build Model

MatchZoo provides many built-in text matching models.

In [70]:
mz.models.list_available()

[matchzoo.models.naive_model.NaiveModel,
 matchzoo.models.dssm_model.DSSMModel,
 matchzoo.models.cdssm_model.CDSSMModel,
 matchzoo.models.dense_baseline_model.DenseBaselineModel,
 matchzoo.models.arci_model.ArcIModel,
 matchzoo.models.knrm_model.KNRMModel,
 matchzoo.models.duet_model.DUETModel,
 matchzoo.models.drmmtks_model.DRMMTKSModel,
 matchzoo.models.drmm.DRMM]

In [71]:
model = mz.models.DenseBaselineModel()

The model is initialized with a hyper parameter table, in which values are partially filled.

In [72]:
print(model.params)

name                          None
model_class                   <class 'matchzoo.models.dense_baseline_model.DenseBaselineModel'>
input_shapes                  None
task                          None
optimizer                     None
with_multi_layer_perceptron   True
mlp_num_units                 256
mlp_num_layers                None
mlp_num_fan_out               None
mlp_activation_func           None


In [73]:
model.params['name'] = 'My First Model'
model.params['mlp_num_units'] = 3
print(model.params)

name                          My First Model
model_class                   <class 'matchzoo.models.dense_baseline_model.DenseBaselineModel'>
input_shapes                  None
task                          None
optimizer                     None
with_multi_layer_perceptron   True
mlp_num_units                 3
mlp_num_layers                None
mlp_num_fan_out               None
mlp_activation_func           None


Use `guess_and_fill_missing_params` to automatically fill-in other hyper parameters. This involves some guessing so the parameter it fills could be wrong. For example, the default task is `Ranking`, and if we do not set it to `Classification` manaully for data packs prepared for classification, then the shape of the model output and the data will mismatch.

In [74]:
model.guess_and_fill_missing_params()
print(model.params)

Parameter "task" set to Ranking Task.
Parameter "input_shapes" set to [(30,), (30,)].
Parameter "optimizer" set to adam.
Parameter "mlp_num_layers" set to 3.
Parameter "mlp_num_fan_out" set to 32.
Parameter "mlp_activation_func" set to relu.
name                          My First Model
model_class                   <class 'matchzoo.models.dense_baseline_model.DenseBaselineModel'>
input_shapes                  [(30,), (30,)]
task                          Ranking Task
optimizer                     adam
with_multi_layer_perceptron   True
mlp_num_units                 3
mlp_num_layers                3
mlp_num_fan_out               32
mlp_activation_func           relu


In [75]:
model.params.completed()

True

With all parameters filled in, we can now build and compile the model.

In [76]:
model.build()
model.compile()
model.backend.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 30)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 30)           0                                            
__________________________________________________________________________________________________
concatenate_11 (Concatenate)    (None, 60)           0           text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dense_99 (Dense)                (None, 3)            183         concatenate_11[0][0]             
__________

For more details about models, consult `matchzoo/tutorials/models.ipynb`.

# Train, Evaluate, Predict

A `DataPack` can `unpack` itself into data that can be directly used to train a MatchZoo model.

In [77]:
x, y = train_data_pack_processed.unpack()
test_x, test_y = test_data_pack_processed.unpack()

In [78]:
model.fit(x, y, batch_size=32, epochs=5)

Epoch 1/5
20347/20347 [==============================] - 2s 121us/step - loss: 1256.6792 - mean_absolute_error: 6.6904
Epoch 2/5
20347/20347 [==============================] - 1s 62us/step - loss: 0.4539 - mean_absolute_error: 0.1395
Epoch 3/5
20347/20347 [==============================] - 1s 58us/step - loss: 0.0652 - mean_absolute_error: 0.1067
Epoch 4/5
20347/20347 [==============================] - 1s 57us/step - loss: 0.0657 - mean_absolute_error: 0.1132
Epoch 5/5
20347/20347 [==============================] - 1s 56us/step - loss: 0.0636 - mean_absolute_error: 0.1107


An alternative to train a model is to use a `DataGenerator`. This might be useful for delaying expensive preprocessing steps or doing real-time data augmentation. For more details about `DataGenerator`, consult `matchzoo/tutorials/data_handling.ipynb`.

In [79]:
data_generator = mz.DataGenerator(train_data_pack_processed, batch_size=32)

In [80]:
model.fit_generator(data_generator, epochs=5, use_multiprocessing=True, workers=4)

Epoch 1/5
636/636 [==============================] - 5s 7ms/step - loss: 0.0621 - mean_absolute_error: 0.1107
Epoch 2/5
636/636 [==============================] - 6s 9ms/step - loss: 0.0595 - mean_absolute_error: 0.1089
Epoch 3/5
636/636 [==============================] - 4s 7ms/step - loss: 0.0644 - mean_absolute_error: 0.1111
Epoch 4/5
636/636 [==============================] - 4s 6ms/step - loss: 0.0646 - mean_absolute_error: 0.1106
Epoch 5/5
636/636 [==============================] - 3s 5ms/step - loss: 0.0624 - mean_absolute_error: 0.1116


In [81]:
model.evaluate(test_x, test_y)

6116/6116 [==============================] - 1s 109us/step


{'loss': 0.04958058035328858, 'mean_absolute_error': 0.08137181649280577}

In [82]:
model.predict(test_x)

array([[ 0.0290747 ],
       [ 0.03049568],
       [ 0.05234022],
       ...,
       [ 0.05234022],
       [-0.01602625],
       [ 0.02186111]], dtype=float32)

# Automation

MatchZoo strives for ease of use, and package `matchzoo.auto` is a perfect example of that.

`matchzoo.auto.prepare` handles interaction among data, model, and preprocessor automatically. For example, some model like `DSSM` have dynamic input shapes based on the result of word hashing. Some models have an embedding layer which dimension is related to the data's vocabulary size. `prepare` takes care of all that and returns properly prepared model, data, and preprocessor for you.

In [83]:
model_ok, train_ok, preprocesor_ok = mz.auto.prepare(
    model=mz.models.DSSMModel(),
    data_pack=train_data_pack[:100]
)
test_ok = preprocesor_ok.transform(test_data_pack, verbose=0)
model_ok.fit(*train_ok.unpack(), batch_size=32)
model_ok.evaluate(*test_ok.unpack())

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 13/13 [00:00<00:00, 2099.17it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 100/100 [00:00<00:00, 2978.23it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 8523/8523 [00:00<00:00, 2028373.41it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 13/13 [00:00<00:00, 3881.96it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 100/100 [00:00<00:00, 2568.20it/s]

Parameter "name" set to DSSMModel.
Parameter "mlp_num_layers" set to 3.
Parameter "mlp_num_units" set to 64.
Parameter "mlp_num_fan_out" set to 32.
Parameter "mlp_activation_func" set to relu.


Epoch 1/1
6116/6116 [==============================] - 1s 113us/step


{'loss': 0.04682399806163918, 'mean_absolute_error': 0.09136102424095159}

For more details about automation, consult `matchzoo/tutorials/automation.ipynb`.

# Full Example

In [50]:
model_classes = [
#     mz.models.DenseBaselineModel,
    mz.models.DSSMModel,
#     mz.models.ArcIModel,
    mz.models.DUETModel,
    mz.models.KNRMModel
]

In [52]:
task = mz.tasks.Ranking(metrics=['mae', 'ap', 'ndcg'])
results = []
for model_class in model_classes:
    print(model_class)
    model = model_class()
    model.params['task'] = task
    model_ok, train_ok, preprocesor_ok = mz.auto.prepare(
        model=model,
        data_pack=train_data_pack[:2000],
        verbose=0
    )
    test_ok = preprocesor_ok.transform(test_data_pack, verbose=0)
    callback = mz.engine.BaseModel.EvaluateOnCall(
        model_ok,
        *test_ok.unpack(),
        valid_steps=1,
        batch_size=1024
    )
    history = model_ok.fit(*train_ok.unpack(), batch_size=32, epochs=30, callbacks=[callback])
    results.append({
        'name': model_ok.params['name'],
        'history': history
    })

<class 'matchzoo.models.dssm_model.DSSMModel'>
Epoch 1/30
2000/2000 [==============================] - 2s 1ms/step - loss: 0.0465 - mean_absolute_error: 0.0688


Validation: loss:0.045543 - mean_absolute_error:0.076757 - average_precision(0):0.109458 - normalized_discounted_cumulative_gain@1(0):0.171429


Epoch 2/30
2000/2000 [==============================] - 0s 221us/step - loss: 0.0448 - mean_absolute_error: 0.0774


Validation: loss:0.045394 - mean_absolute_error:0.088274 - average_precision(0):0.088499 - normalized_discounted_cumulative_gain@1(0):0.117460


Epoch 3/30
2000/2000 [==============================] - 0s 230us/step - loss: 0.0439 - mean_absolute_error: 0.0750


...

Validation: loss:67.386414 - mean_absolute_error:6.560674 - average_precision(0):0.086512 - normalized_discounted_cumulative_gain@1(0):0.103175


Epoch 29/30
2000/2000 [==============================] - 5s 2ms/step - loss: 3.1524 - mean_absolute_error: 1.2739


Validation: loss:71.351753 - mean_absolute_error:6.762890 - average_precision(0):0.087010 - normalized_discounted_cumulative_gain@1(0):0.104762


Epoch 30/30
2000/2000 [==============================] - 4s 2ms/step - loss: 3.5766 - mean_absolute_error: 1.3456


Validation: loss:64.831316 - mean_absolute_error:6.433554 - average_precision(0):0.086082 - normalized_discounted_cumulative_gain@1(0):0.095238


In [53]:
import bokeh
from bokeh.io import output_notebook, push_notebook
from bokeh.layouts import column
from bokeh.models.glyphs import Line
from bokeh.models.tools import HoverTool
from bokeh.plotting import figure, show, Figure

In [56]:
charts = {
    metric: figure(
        title=str(metric),
        sizing_mode='scale_width',
        width=800, height=400
    ) for metric in results[0]['history'].history.keys()
}
for metric, sub_chart in charts.items():
    lines = {}
    for result, color in zip(results, bokeh.palettes.Category10[10]):
        x = result['history'].epoch
        y = result['history'].history[metric]
        lines[result['name']] = sub_chart.line(
            x, y, color=color, line_width=2, alpha=0.5, legend=result['name'])
output_notebook()
show(column(*charts.values()))

Loading BokehJS ...